In [1]:
#Importa as bibliotecas essenciais
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import tifffile
import skimage as ski
from skimage.filters import threshold_multiotsu
import cv2
from skimage import data, io
from skimage import exposure # Módulo onde o CLAHE está

In [2]:
file_path = '/home/ronald/Schistossoma/original_data/30dias'#original_data/120dias/'

path_destiny = '/home/ronald/Schistossoma/processed_files_30dias'

file_path2 = '/home/ronald/Schistossoma/original_data/60dias'#original_data/120dias/'

path_destiny2 = '/home/ronald/Schistossoma/processed_files_60dias'

In [3]:
def rescale(data, min_value, max_value):
  try:
    data_min, data_max = data.min(), data.max()
    data_scaled = np.round((data - data_min) * (max_value - min_value) / (data_max - data_min) + min_value)
    data_scaled[data_scaled > max_value] = max_value          #Garante que após o arredondamento nenhum valor seja superior ao valor máximo definido
    return data_scaled.astype(np.uint16)
  except Exception as e:
    print('Ocorreu um erro inesperado: {}'.format(e))
    return None

def calculate_treshold_and_variance_between_classes(image):
  '''Calcula o threshold que maximiza a variância entre classes (método de Otsu),
     assim como a variância máxima, para duas classes somente'''
  intensity, count = np.unique(image, return_counts=True)

  n = len(intensity)

  probability = count/count.sum()

  cumulative_probability = probability.cumsum()

  E_i = intensity * probability

  partial_weighted_sum = E_i.cumsum()

  global_mean = E_i.sum()

  mean_C1 = np.zeros_like(intensity, dtype=float)

  cumulative_probability_C1 = np.zeros_like(intensity, dtype=float)

  mean_C0 = partial_weighted_sum/cumulative_probability

  mean_C1[:-1] = (partial_weighted_sum[-1] - partial_weighted_sum[:-1]) / (1 - cumulative_probability[:-1])

  cumulative_probability_C1[:-1] = 1 - cumulative_probability[:-1]

  variance_between_classes = cumulative_probability*(global_mean - mean_C0)**2+cumulative_probability_C1*(global_mean-mean_C1)**2

  indice_maior_valor = np.argmax(variance_between_classes)

  return intensity[indice_maior_valor], variance_between_classes[indice_maior_valor]


def find_best_parameters(image):
  clip_limits_to_try = np.arange(0.01,0.21,0.01)
  kernel_sizes_to_try = [(8,8),(12,12),(16,16),(20,20),(24,24),(28,28),(32,32)]
  clip_limits_index, kernel_sizes_index = np.meshgrid(np.arange(len(clip_limits_to_try)), np.arange(len(kernel_sizes_to_try)))
  clip_limits_index, kernel_sizes_index = clip_limits_index.flatten(), kernel_sizes_index.flatten()
  clip_limits = clip_limits_to_try[clip_limits_index]
  kernel_sizes = [kernel_sizes_to_try[i] for i in kernel_sizes_index]
  results = [
    calculate_treshold_and_variance_between_classes(
        rescale(exposure.equalize_adapthist(image, clip_limit=clip_limits[i], kernel_size=kernel_sizes[i]), min(image.ravel()), max(image.ravel()))
    )
    for i in range(len(clip_limits))
  ]
  thresholds, variances = zip(*results)
  return int(thresholds[np.argmax(variances)])


In [4]:
def interest_class(figure, fator_multiplicativo):
  blur = cv2.GaussianBlur(figure, (65,65), 0)
  threshold=find_best_parameters(blur)
  SHG_class = figure.copy()
  for row in SHG_class:
    for i in range(len(row)):
      if row[i] < threshold:
        row[i] = 11*fator_multiplicativo
  return SHG_class



def processar_imagens(diretorio_origem, diretorio_destino):
  archives_names = [f for f in os.listdir(diretorio_origem)
                     if os.path.isfile(os.path.join(diretorio_origem, f)) 
                     and f.lower().endswith('.tif')
                     and 'shg' in f]                             #Arquivos ".tif" no diretório
  os.makedirs(diretorio_destino, exist_ok=True)
  os.makedirs(diretorio_destino+'/SHG', exist_ok=True)
  count = 1
  for figure in archives_names:
    print('{}/{}'.format(count,len(archives_names)))
    fator_multiplicativo = int([a for a in ((figure.strip()).replace('.tif','')).split('-') if 'acc' in a.lower()][0].lower().replace('acc',''))
    image = ski.io.imread(os.path.join(diretorio_origem, figure))
    results = list()
    for c in range(image.shape[2]):
      SHG_class = interest_class(image[:, :, c], fator_multiplicativo)
      results.append(SHG_class)
      if c==1:
        base_name = os.path.splitext(figure)[0]  # Nome do arquivo sem extensão
        original_png = f"{base_name}_layer{c}_original.png"
        processed_png = f"{base_name}_layer{c}_processed.png"
        # Normaliza a camada original para 0–255 e converte para uint8
        original_layer_scaled = ((image[:, :, c] / 4095) * 255).astype(np.uint8)
        processed_layer_scaled = ((SHG_class / 4095) * 255).astype(np.uint8)
        #Salva as imagens PNG
        ski.io.imsave(os.path.join(diretorio_destino, original_png), original_layer_scaled)
        ski.io.imsave(os.path.join(diretorio_destino, processed_png), processed_layer_scaled)
    results_array = np.stack(results)
    # Converter para shape (T=1, Z=3, C=1, Y=512, X=512)
    stack_4d = results_array[:, np.newaxis, :, :]             # (Z, 1, Y, X)    #Adiciona eixo do canal ao stack
    stack_5d = stack_4d[np.newaxis, ...]              # (1, Z, 1, Y, X)         #Adiciona eixo do tempo ao stack
    # Salvar como multi-TIFF
    repository_and_image_path = diretorio_destino+'/SHG/'+ figure

    tifffile.imwrite(
    repository_and_image_path,
    stack_5d,
    photometric='minisblack',  # escala de cinza (0=preto, 255=branco)
    metadata={'axes': 'TZCYX'},  # convenção usada por tifffile
    resolution=(72, 72),       # DPI (igual ao do arquivo que você enviou)
    imagej=True                # Adiciona cabeçalhos compatíveis com ImageJ/Zeiss
    )
    count += 1

In [ ]:
processar_imagens(file_path, path_destiny)

1/68
2/68
3/68
4/68
5/68
6/68
7/68
8/68
9/68
10/68
11/68
12/68
13/68
14/68
15/68
16/68
17/68
18/68


In [ ]:
processar_imagens(file_path2, path_destiny2)

In [ ]:
file_path3 = '/home/ronald/Schistossoma/original_data/control'#original_data/120dias/'

path_destiny3 = '/home/ronald/Schistossoma/processed_files_control'

processar_imagens(file_path3, path_destiny3)